### Optimization - QCQP, barrier method

##### 1.다음과 같은 QCQP 형태의 최적화 문제를 고려.
##### $\text{minimize}_x \quad \frac{1}{2}x^TQx + p^Tx $, 
##### $\text{s.t} \quad \frac{1}{2}x^TQ_ix + p_{i}^{T}x \leq 0  \quad i = 1, \dots , m$

##### 위의 부등제약 조건을 아래와 같이 지시함수 형태로 포함, 여전히 boundary를 가지고 잇고, 미분 불가능 하기 때문에 계산상 어려움 존재 
##### $C = \{x:\frac{1}{2}x^TQ_ix + p_{i}^{T}x \leq 0 \quad i = 1, \dots , m\}$

##### $\text{minimize}_x \quad \frac{1}{2}x^TQx + p^Tx + I_c(x)$ 
##### 해결) barrier function을 이용해 boundary 포함하지않고, 미분가능 하게해서 newton's method 적용

##### 2.logarithmic barrier function 구하기
##### $h_i(x)=\frac{1}{2}x^TQ_ix + p_{i}^{T}x 라고 하자.$ -> convex이고, 두번 미분가능
##### $\phi(x)= -\sum_{i=1}^{m}log(-h_i(x))$ -> logarithmic barrier function

##### 3.barrier function을 적용 시킨 최적화 문제.
##### $\text{minimize}_x \quad t(\frac{1}{2}x^TQx + p^Tx) + \phi(x)$ 
##### 이와 같이 근사해서 정의된 문제를 newton's method로 푸는 방법을 barrier method라고 함.

In [4]:
import numpy as np
import matplotlib.pyplot as plt

alpha = 0.1
beta = 0.7
mu = 5

n = 20
d = 15

eps = 10e-6

lambd = 10

In [23]:
def function_true(x, Q, p):
    """ function = 1/2*xTQx + pTx
    st. 1/2*xTQ[i]x + pT[i]x <= 0, i = 1, ... ,m
    """
    return 1/2 * np.dot(x.T, np.dot(Q, x)) + np.dot(p.T, x)

def function(x, Q, p, t0):
    """ function = t(1/2*xTQx + pTx) + phi(x)
    Q, p : 쿼드라틱 폼의 메트릭스
    """
    return t0 * (1/2 * np.dot(x.T, np.dot(Q, x)) + np.dot(p.T, x)) - sum([np.log(- 1/2 * np.dot(x.T, np.dot(Q[i], x)) - np.dot(p[i], x)) for i in range(Q.shape[0])])

##### log barrier 계산.
##### $h_i(x)=\frac{1}{2}x^TQ_ix + p_{i}^{T}x$ 
##### $\phi(x)= -\sum_{i=1}^{m}log(-h_i(x))$ 
##### gradient: $\bigtriangledown \phi(x) = - \sum_{i=1}^{m} \frac{1}{h_i(x)}\bigtriangledown h_i(x)$
##### Hessian: $\bigtriangledown^2 \phi(x) = \sum_{i=1}^{m} \frac{1}{h_i(x)^2}\bigtriangledown h_i(x)\bigtriangledown h_i(x)^T - \sum_{i=1}^{m} \frac{1}{h_i(x)}\bigtriangledown^2 h_i(x) $

In [7]:
def line_search(Q, p, x, t, df, dx, t0):
    """
    Backtracking line shearch, 해보정법
    """
    if function(x + t*dx, Q, p, t0) <= (function(x, Q, p, t0) + alpha*t*np.dot(df.T, dx)):
        return x + t*dx
    else:
        return line_search(Q, p, x, beta*t, df, dx , t0)

In [10]:
def centering_step(Q, p, t, x0, eps, numb_iter = 0): 
    df = t*(np.dot(Q, x0) + p) + \
        sum([((np.dot(Q[i], x0) + p[i]) / (- 1/2 * np.dot(x0.T, np.dot(Q[i], x0)) - np.dot(p[i], x0))) for i in range(Q.shape[0])])
    d2f = t*Q + \
        sum ([(((Q[i]*(- 1/2 * np.dot(x0.T, np.dot(Q[i], x0)) - np.dot(p[i], x0))) + ((np.dot(Q[i], x0) + p[i]) ** 2)) / (- 1/2 * np.dot(x0.T, np.dot(Q[i], x0)) - np.dot(p[i], x0))**2) for i in range(Q.shape[0])])
    dx = -1*np.dot(np.linalg.inv(d2f), df)
    l2 = np.dot(df.T, np.dot(np.linalg.inv(d2f), df))
    if l2/2 <= eps:
        return x0, numb_iter
    v1 = line_search(Q, p, x0, t=1, df=df, dx=dx, t0=t)
    return centering_step(Q, p, t, x1, eps, numb_iter+1)

In [11]:
def barr_method_inter(Q, p, x0, eps, t, mu, numb_iter=0, numb_newton = [], x_seq = [], f_seq_true= []):
    '''
    Fonction récursif de la méthode de la barrière.
    '''
    x_center, numb_iter_inter = centering_step(Q, p, t, x0, eps)
    numb_newton.append(numb_iter)
    x_seq.append(x_center)
    f_seq_true.append(function_true(x_center, Q, p)[0][0])
    if Q.shape[0]/t < eps:
        return x_center, numb_newton, x_seq, f_seq_true
    else:
        t = mu*t
        return barr_method_inter(Q, p, x0, eps, t, mu, numb_iter+numb_iter_inter, numb_newton, x_seq, f_seq_true)


In [12]:
def barr_method(Q, p, x0, eps, mu):
    '''
    Fonction de la méthode de la barrière.
    '''
    numb_newton = [0]
    v_seq = [x0]
    f_seq_true = [function_true(x0, Q, p)[0][0]]
    return barr_method_inter(Q, p, x0, eps, 1, mu, 0, numb_newton, v_seq, f_seq_true)

In [22]:
x = np.random.rand(n,d)
p = np.random.rand(n,d)
Q = np.eye(n)
x0 = np.zeros((n,d))

mu_list = [2, 5, 10, 15, 20, 25, 30, 40, 50, 70, 85, 100, 130, 170, 200, 250]
mu_list_restricted = [2, 5, 15, 50, 100, 200]
w_center_list = []
f_true_list = []

## Calcul et plot des résultats
plt.figure()
for mu in mu_list:
    x_center, numb_newton, x_seq, f_seq_true = barr_method(Q, p, x0, eps, mu)
    w_center = np.linalg.lstsq(x,-x_center-y)[0]
    w_center_list.append(w_center)
    f_true_list.append(f_seq_true[-1])
    if mu in mu_list_restricted:
        plt.step(numb_newton, f_seq_true - f_seq_true[-1], label='mu = '+str(mu))
plt.legend()
plt.ylabel("Norm of f(mu) - f(mu)*")
plt.xlabel("Nombre d'étapes (de centering step)")
plt.semilogy()
plt.show()

## Plot de w en fonction de mu
mu_min = np.argmin(f_true_list)
plt.figure()
w_diff_norm = [np.linalg.norm(w-w_center_list[mu_min]) for w in w_center_list]
plt.plot(mu_list, w_diff_norm)
plt.ylabel("Norm of w(mu) - w(mu*)")
plt.xlabel("mu")
plt.show()

ValueError: shapes (15,20) and (15,) not aligned: 20 (dim 1) != 15 (dim 0)

<Figure size 432x288 with 0 Axes>

##### $min_x \quad tf(x) + \phi(x)$
##### $s.t \quad Ax = b$